# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Julia Hernández Cárdenas  |
| **Fecha**      | 15/09/2025  |
| **Expediente** | 739208  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

**¿Por qué?**

Se utiliza para tratar de evitar que el modelo esté "overfit". O sea, tratar de ver si el modelo puede funcionar para cualquier grupo de datos, no únicamente con los que se está entrenando. 

**Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?**

A pesar de que tendría sentido utilizar todos los datos, el modelo terminaría teniendo un problema de "overfitting", haciendo que no sea preciso a la hora de utilizar datos nuevos, que no hayan sido utilizados para entrenar este modelo. 

La idea de hacer estos modelos es poder utilizarlos para cualquier conjunto de datos, y si éste está sobreajustado, no sería útil tenerlo, ya que los resultados no serían correctos.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
motor = pd.read_excel("Motor Trend Car Road Tests.xlsx")
motor.dropna()
motor.drop_duplicates()
motor = motor.drop(columns = ["model"])

In [3]:
x = motor.drop(columns=["mpg"])
y = motor["mpg"]

In [4]:
n = len(motor)
valores_mse = []
for i in range(n):
    x_train = x.drop(i).to_numpy()
    y_train = y.drop(i).to_numpy()
    x_test = x.iloc[i].to_numpy().reshape(1, -1)
    y_test = y.iloc[i]
    
    model = LinearRegression()
    model.fit(x_train, y_train)
    
    y_pred = model.predict(x_test)
    
    mse = mean_squared_error([y_test], y_pred)
    valores_mse.append(mse)

print("MSE promedio:", np.mean(valores_mse))
print("Desviación estándar:", np.std(valores_mse))

MSE promedio: 12.181558006901964
Desviación estándar: 17.06739987188858


In [5]:
valores_mse

[5.258855796629032,
 2.453996713086818,
 20.550493416882755,
 0.04432910713964311,
 1.5811555159760176,
 10.11862548887538,
 0.01637404154847087,
 8.081250710389417,
 39.47018347964965,
 0.7706327722901085,
 4.955694234215864,
 10.224730198354456,
 4.430129331402683,
 0.4878127712478519,
 6.823844575809502,
 0.6027506975547144,
 36.810178609958804,
 31.759855581210513,
 1.0632275418057964,
 32.71328618702714,
 14.30717368271277,
 3.405354649998888,
 9.408007649079883,
 0.00010350039803663976,
 9.962439534547652,
 1.3412208919431396,
 0.1648006408222328,
 23.600206502249318,
 83.11457643385218,
 0.00010271237184903523,
 8.785533367966966,
 17.502929883865335]

**ESCALANDO**

In [6]:
x2 = motor.drop(columns=["mpg"])
y2 = motor["mpg"]

In [7]:
n = len(motor)
valores_mse2 = []
for i in range(n):
    x2_train = x2.drop(i).to_numpy()
    y2_train = y2.drop(i).to_numpy()
    x2_test = x2.iloc[i].to_numpy().reshape(1, -1)
    y2_test = y2.iloc[i]
    
    scaler = StandardScaler().fit(x2_train)
    x_scaled_train = scaler.transform(x2_train)
    x_scaled_test = scaler.transform(x2_test)
    
    model = LinearRegression()
    model.fit(x_scaled_train, y2_train)
    
    y_pred2 = model.predict(x_scaled_test)
    
    mse2 = mean_squared_error([y2_test], y_pred2)
    valores_mse2.append(mse2)

print("MSE promedio:", np.mean(valores_mse2))
print("Desviación estándar:", np.std(valores_mse2))

MSE promedio: 12.18155800690193
Desviación estándar: 17.067399871888554


In [8]:
valores_mse2

[5.258855796629032,
 2.453996713086796,
 20.55049341688269,
 0.04432910713964311,
 1.5811555159760176,
 10.11862548887538,
 0.0163740415484736,
 8.081250710389499,
 39.47018347964965,
 0.7706327722901085,
 4.955694234215864,
 10.22473019835441,
 4.430129331402705,
 0.4878127712478618,
 6.823844575809344,
 0.6027506975546868,
 36.81017860995895,
 31.759855581210353,
 1.0632275418058037,
 32.713286187027215,
 14.307173682712742,
 3.405354649998901,
 9.408007649079861,
 0.00010350039803660362,
 9.962439534547741,
 1.3412208919431232,
 0.1648006408222357,
 23.60020650224925,
 83.11457643385192,
 0.00010271237184867518,
 8.785533367966272,
 17.502929883865395]

*No cambia practicamente nada porque las Y no están siendo escaladas

**Interpreta.**

A pesar de que el modelo está esencialmente bien y predice bien, no es confiable y no estaría bien afirmar que se puede generalizar porque la desviación estándar es demasiado alta, indicando que en algunos modelos, el MSE promedio fue muy preciso pero en otros se equivocó mucho. O sea, este modelo predice muy bien para ciertos autos, pero para otros no 

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [9]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [10]:
housing_new = np.random.permutation(len(X))

X_shuffled = X[housing_new]
y_shuffled = y[housing_new]

In [14]:
n = len(X_shuffled)
k = 10
fold_size = n // k

valores_mse = []

for i in range(k):
    start = i * fold_size
    end = (i + 1) * fold_size
    
    x_test = X_shuffled[start:end]
    y_test = y_shuffled[start:end]
    
    x_train = np.concatenate([X_shuffled[:start], X_shuffled[end:]])
    y_train = np.concatenate([y_shuffled[:start], y_shuffled[end:]])
    
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    
    y_pred = lr.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    valores_mse.append(mse)

print("MSE promedio:", np.mean(valores_mse))
print("Desviación estándar:", np.std(valores_mse))

MSE promedio: 0.528790281993409
Desviación estándar: 0.022931070333301374


Interpreta.

Diría que este modelo si es confiable y preciso, ya que, la desviación estándar es muy baja, indicando que no hay mucha variación entre las diferentes 10 pruebas.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3